Code to process last 5 year FnO transaction data

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from datetime import timedelta

# Load API Key and download directory from config file
CONFIG_FILE = "../config/file-config.json"

def load_config(config_path):
    """Load configuration from a JSON file."""
    with open(config_path, "r") as file:
        return json.load(file)

# Load config values
config = load_config(CONFIG_FILE)
print(config)
# print(config)
raw_dir = config["tradebook"]["raw_dir"]
processed_dir = config["tradebook"]["processed_dir"]
print("raw_dir :", raw_dir)

df_pnl_file_path = f'{raw_dir}/pnl-ZH5601-2020-2025.xlsx'
df_ledger_file_path = f'{raw_dir}/ledger-ZH5601-2019-2025.csv'
df_tradebook_file_path = f'{raw_dir}/tradebook-ZH5601-FO-2020.csv'

df_pnl = pd.read_excel(df_pnl_file_path, index_col=0, parse_dates=True)
df_pnl.reset_index(inplace=True)
print("Printing df_pnl columns : ", df_pnl.columns)

df_ledger = pd.read_csv(df_ledger_file_path, index_col=0, parse_dates=True)
df_ledger.reset_index(inplace=True)
print("Printing df_ledger columns : ", df_ledger.columns)

# df_tradebook = pd.read_csv(df_tradebook_file_path, index_col=0, parse_dates=True)
# df_tradebook.reset_index(inplace=True)
# print("Printing df_tradebook columns : ", df_tradebook.columns)


{'ledger': {'raw_dir': '../data/raw/', 'processed_dir': '../data/processed/'}, 'pnl': {'raw_dir': '../data/raw/', 'processed_dir': '../data/processed/'}, 'tradebook': {'raw_dir': '../data/raw/', 'processed_dir': '../data/processed/'}}
raw_dir : ../data/raw/
Printing df_pnl columns :  Index(['index', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')
Printing df_ledger columns :  Index(['particulars', 'posting_date', 'cost_center', 'voucher_type', 'debit',
       'credit', 'net_balance'],
      dtype='object')


/var/folders/gy/xs43v__d0mn25qy5kzxrnkrm0000gn/T/ipykernel_8899/2242009098.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ledger = pd.read_csv(df_ledger_file_path, index_col=0, parse_dates=True)


Merge all files into consolidated file

In [4]:
# Define the base file path and the list of file suffixes (23, 24, 28, 31, 53, 54, 55, 56= empty)
base_file_path = f'{raw_dir}tradebook-ZH5601-FO-'
file_suffixes = [2020, 2021, 2022, 2023, 2024, 2025]
merged_df_total = pd.DataFrame()

# Iterate over the file suffixes and merge the data
for suffix in file_suffixes:
    file_path = f'{base_file_path}{suffix}.csv'
    # print("Processing file_path :", file_path)
    df_tradebook = pd.read_csv(file_path)
    #, parse_dates=['trade_date'], index_col='trade_date')
    # print("df_ned DataFrame shape :", df_ned.shape[0])
    # df_tradebook.reset_index(inplace=True)
    # print(df_tradebook.columns)

    if(merged_df_total.empty):
        merged_df_total = df_tradebook.copy()
    else:
        # Only select columns that match the existing file
        df_tradebook_subset = df_tradebook[merged_df_total.columns]
        # Now concatenate with matching columns
        merged_df_total = pd.concat([merged_df_total, df_tradebook_subset], axis=0, ignore_index=True)

file_path = os.path.join(processed_dir, f"tradebook-consolidated-data-2020-2025.csv")  # Change extension as needed
with open(file_path, "wb") as file:
    merged_df_total.to_csv(file_path, index=False)
    print(f"Consolidated data file saved at : ", file_path)
print(merged_df_total.head(3))

Consolidated data file saved at :  ../data/processed/tradebook-consolidated-data-2020-2025.csv
                  symbol  isin  trade_date exchange segment  series  \
0  HINDUNILVR20DEC2380CE   NaN  2020-12-14      NSE      FO     NaN   
1  HINDUNILVR20DEC2380CE   NaN  2020-12-14      NSE      FO     NaN   
2  HINDUNILVR20DEC2380CE   NaN  2020-12-14      NSE      FO     NaN   

  trade_type  auction  quantity  price   trade_id          order_id  \
0        buy    False     300.0  50.00  500283425  1300000019296515   
1       sell    False     300.0  45.15  500591592  1300000048651812   
2        buy    False     300.0  46.30  500599413  1300000048788254   

  order_execution_time expiry_date  
0  2020-12-14T11:31:12  2020-12-31  
1  2020-12-14T15:25:41  2020-12-31  
2  2020-12-14T15:28:14  2020-12-31  


Link instrument buy with instrument sell and calculate profile
summarize instrument in understandable view
link profile / loss with index move data
perform prediction

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Filter the DataFrame to include only the volume columns
df_volume = merged_df_total[volume_columns]

# Plot the volume columns
plt.figure(figsize=(15, 10))
for col in df_volume.columns:
    plt.plot(df_volume.index, df_volume[col], label=col)

plt.xlabel('Time')
plt.ylabel('Volume')
plt.title('Volume Variables Over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(df_filtered['Total_Flow'], df_filtered['wind_speed_10m'], s=1, alpha=0.5)  # Adjust 's' for point size
plt.xlabel('Total_Flow')
plt.ylabel('Wind Speed')
plt.title('Scatter Plot of Total_Flow vs Wind Speed')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(df_filtered['wind_speed_10m'], df_filtered['production_wind'], s=5, alpha=0.5)  # Adjust 's' for point size
plt.xlabel('Wind Speed')
plt.ylabel('Wind Energy Production')
plt.title('Scatter Plot of Wind Speed vs Wind Energy')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np

# Drop rows with missing values in 'Price' or 'Load' columns
df_filtered_ws = df_filtered[['production_wind', 'wind_speed_10m']].dropna()

# Calculate the correlation coefficient
r = np.corrcoef(df_filtered_ws['production_wind'], df_filtered_ws['wind_speed_10m'])
print("Correlation coefficient matrix:")
print(r)

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and a set of subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Scatter plot for 'diffuse_radiation'
axs[0].scatter(df_filtered['diffuse_radiation'], df_filtered['production_solar'], s=5, alpha=0.5)
axs[0].set_xlabel('Diffuse Radiation')
axs[0].set_ylabel('Solar Energy Production')
axs[0].set_title('Scatter Plot of Diffuse Radiation vs Solar Energy Production')
axs[0].grid(True)

# Scatter plot for 'direct_normal_irradiance'
axs[1].scatter(df_filtered['direct_normal_irradiance'], df_filtered['production_solar'], s=5, alpha=0.5)
axs[1].set_xlabel('Direct Normal Irradiance')
axs[1].set_ylabel('Solar Energy Production')
axs[1].set_title('Scatter Plot of Direct Normal Irradiance vs Solar Energy Production')
axs[1].grid(True)

# Scatter plot for 'shortwave_radiation'
axs[2].scatter(df_filtered['shortwave_radiation'], df_filtered['production_solar'], s=5, alpha=0.5)
axs[2].set_xlabel('Shortwave Radiation')
axs[2].set_ylabel('Solar Energy Production')
axs[2].set_title('Scatter Plot of Shortwave Radiation vs Solar Energy Production')
axs[2].grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Select the desired columns
selected_columns = [
    'Load', 'Price', 'Total_Flow', 'temperature_2m', 'apparent_temperature', 
    'cloud_cover', 'wind_speed_10m', 'diffuse_radiation', 'direct_normal_irradiance', 
    'shortwave_radiation', 'wind_speed_100m', 'location', 'production_wind', 
    'production_solar', 
    'production_heatpump', 
    'production_cofiring', 
    'production_geothermal', 
    'production_other', 
    'production_waste', 
    'production_biooil', 
    'production_biomass', 
    'production_wood', 
    'production_windoffshore', 
    'production_fossilgaspower', 
    'production_fossilhardcoal', 
    'production_nuclear', 
    'production_wastepower', 
    'production_windoffshoreB', 
    'production_biomasspower', 
    'production_otherpower', 
    'production_electricitymix', 
    'production_CHP_total', 
    'production_solarthermal', 
    'production_allconsuminggas'
]

# Filter the DataFrame to include only the selected columns
df_selected = df_filtered[selected_columns]

# Select only the numerical columns
df_numeric = df_selected.select_dtypes(include=['number'])

# Calculate the correlation matrix
corr_matrix = df_numeric.corr()

# Create an interactive heatmap with Plotly
fig = px.imshow(corr_matrix, 
                labels=dict(color="Correlation"), 
                x=corr_matrix.columns, 
                y=corr_matrix.columns,
                color_continuous_scale='RdBu_r',
                zmin=-1, zmax=1)

fig.update_layout(title='Correlation Matrix', width=800, height=800)

# Save the figure as an HTML file
fig.write_html("correlation_matrix.html")

# Display a message to the user
print("The interactive correlation matrix has been saved as 'correlation_matrix.html'. Open this file in your browser to view it.")

In [ ]:
# Define a color map for the variables
colors = plt.cm.get_cmap('tab10', 5)  # Use a colormap with 5 distinct colors

# Plot the scatter plot with different colors for each variable
plt.figure(figsize=(10, 5))
plt.scatter(df_filtered['Flow_DE'], df_filtered['wind_speed_10m'], s=1, alpha=0.5, color=colors(0), label='Flow_DE')
plt.scatter(df_filtered['Flow_DK'], df_filtered['wind_speed_10m'], s=1, alpha=0.5, color=colors(1), label='Flow_DK')
plt.scatter(df_filtered['Flow_BE'], df_filtered['wind_speed_10m'], s=1, alpha=0.5, color=colors(2), label='Flow_BE')
plt.scatter(df_filtered['Flow_GB'], df_filtered['wind_speed_10m'], s=1, alpha=0.5, color=colors(3), label='Flow_GB')
plt.scatter(df_filtered['Flow_NO'], df_filtered['wind_speed_10m'], s=1, alpha=0.5, color=colors(4), label='Flow_NO')

plt.xlabel('Flow')
plt.ylabel('Wind Speed')
plt.title('Scatter Plot of Flow vs Wind Speed')
plt.legend(title='Variable')
plt.grid(True)
plt.show()

In [ ]:
nan_totalflow = df_filtered['Total_Flow'].isna().sum()
print(nan_totalflow)
nan_temp = df_filtered['temperature_2m'].isna().sum()
print(nan_temp)
nan_cloud = df_filtered['cloud_cover'].isna().sum()
print(nan_cloud)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Drop rows with missing values in the relevant columns
df_filtered_regr = df_filtered[['Price', 'Load', 'Total_Flow', 'temperature_2m', 'apparent_temperature', 'cloud_cover', 'wind_speed_10m', 'diffuse_radiation', 
    'direct_normal_irradiance', 'shortwave_radiation', 'wind_speed_100m', 'production_all', 'production_wind', 'production_solar', 'production_heatpump', 
    'production_cofiring', 'production_geothermal', 'production_other', 'production_waste', 'production_biooil', 'production_biomass', 'production_wood', 
    'production_windoffshore', 'production_fossilgaspower', 'production_fossilhardcoal', 'production_nuclear', 'production_wastepower', 'production_windoffshoreB', 
    'production_biomasspower', 'production_otherpower', 'production_electricitymix', 'production_CHP_total', 'production_solarthermal', 
    'production_allconsuminggas']].dropna()

# Check if the DataFrame is empty
if df_filtered_regr.empty:
    print("The DataFrame is empty after dropping rows with missing values.")
else:
    # Define the features (X) and the target (y)
    X = df_filtered_regr[['Load', 'Total_Flow', 'temperature_2m', 'apparent_temperature', 'cloud_cover', 'wind_speed_10m', 'diffuse_radiation', 'direct_normal_irradiance', 
    'shortwave_radiation', 'wind_speed_100m', 'production_all', 'production_wind', 'production_solar', 'production_heatpump', 'production_cofiring', 'production_geothermal', 
    'production_other', 'production_waste', 'production_biooil', 'production_biomass', 'production_wood', 'production_windoffshore', 'production_fossilgaspower', 
    'production_fossilhardcoal', 'production_nuclear', 'production_wastepower', 'production_windoffshoreB', 'production_biomasspower', 'production_otherpower', 
    'production_electricitymix', 'production_CHP_total', 'production_solarthermal', 'production_allconsuminggas']]
    y = df_filtered_regr['Price']

    # Train the linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Make predictions on the same dataset
    y_pred = model.predict(X)

    # Evaluate the model
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)

    print(f"Mean Squared Error: {mse}")
    print(f"R^2 Score: {r2}")

    # Get the coefficients of the model
    coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
    print(coefficients)

        # Sort the coefficients by absolute value
    coefficients['abs_coefficient'] = coefficients['Coefficient'].abs()
    coefficients = coefficients.sort_values(by='abs_coefficient', ascending=True)

    # Plot the coefficients
    plt.figure(figsize=(12, 8))
    plt.barh(coefficients.index, coefficients['Coefficient'], color='skyblue')
    plt.xlabel('Coefficient Value')
    plt.ylabel('Features')
    plt.title('Feature Importance in Predicting Price')
    plt.grid(True)
    plt.show()



In [ ]:
df_filtered_regr.head